In [1]:
import archs4py as a4
import pandas as pd
import numpy as np

import os
import re
import random
import tqdm


In [2]:
gg = ">ENST00000631435.1 cdna chromosome:GRCh38:CHR_HSCHR7_2_CTG6:142847306:142847317:1 gene:ENSG00000282253.1 gene_biotype:TR_D_gene transcript_biotype:TR_D_gene gene_symbol:TRBD1 description:T cell receptor beta diversity 1 [Source:HGNC Symbol;Acc:HGNC:12158]"
gg.split(" ")[0].split(".")[0].replace(">", "")

'ENST00000631435'

In [2]:
files = os.listdir("data")

def parse_transcripts(file_path):
    sequences = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        i = 0
        while i < len(lines):
            if lines[i].startswith('>'):
                #transcript_id = re.search(r'>(\S+)', lines[i]).group(1)
                #transcript_id = re.sub(r'\.\d+', '', transcript_id)
                transcript_id = lines[i].split(" ")[0].split(".")[0].replace(">", "")
                sequence = ''
                i += 1
                while i < len(lines) and not lines[i].startswith('>'):
                    sequence += lines[i].strip()
                    i += 1
                sequences[transcript_id] = sequence
    return sequences

file_path = '../tempest/files/Homo_sapiens.GRCh38.cdna.all.fa'  # Replace with the actual file path
#file_path = 'Homo_sapiens.GRCh38.cdna.all.fa'  # Replace with the actual file path
transcripts = parse_transcripts(file_path)
print(len(list(transcripts.keys())))

205541


In [6]:
def extract_subsequence(transcripts, transcript_id, length):
    if transcript_id in transcripts:
        sequence = transcripts[transcript_id]
        seq_length = len(sequence)
        if len(sequence) >= length:
            start_position = random.randint(0, seq_length - length)
            subsequence = sequence[start_position:start_position + length]
            return subsequence
        else:
            #print(f"Transcript {transcript_id} is shorter than the requested length.")
            return ""
    else:
        #print(f"Transcript {transcript_id} not found.")
        #print(transcript_id)
        return " "

def mutate_sequence(sequence, mutation_probability):
    mutated_sequence = ''
    for nucleotide in sequence:
        if random.random() < mutation_probability:
            mutated_nucleotide = random.choice('ACTG'.replace(nucleotide, ''))
            mutated_sequence += mutated_nucleotide
        else:
            mutated_sequence += nucleotide
    return mutated_sequence

files = os.listdir("data")
files = [f for f in os.listdir("data") if f != '.DS_Store']
counts = pd.read_csv("data/"+files[0], sep="\t")

reads = []
errors = 0
not_found = 0
for i in tqdm.tqdm(range(counts.shape[0])):
    t = counts.iloc[i, 0]
    count = counts.iloc[i, 1]
    for c in range(count):
        read = extract_subsequence(transcripts, t, 100)
        if len(read) > 1:
            read = mutate_sequence(read, 0.00)
            reads.append(read)
        elif len(read) == 1:
            not_found += 1
        else:
            errors += 1

with open("syndata.fastq", "w") as f:
    counter = 0
    qual = 'J'*100
    for r in reads:
        counter+=1
        f.write(f"@syn.{counter} xx length=100\n")
        f.write(f"{r}\n")
        f.write(f"+syn.{counter} xx length=100\n")
        f.write(f"{qual}\n")

100%|██████████| 271742/271742 [01:39<00:00, 2732.34it/s] 


In [8]:
!"target/release/tempest" -f "../synthetic/syndata.fastq" -t 6 -o "output/counts_syn.tsv" -x "output/syn.idx"

zsh:1: no such file or directory: target/release/tempest


set()

In [90]:
print(len(reads))
print(counts.iloc[:,1].sum())

11964787
11964787


In [3]:
import faiss
import numpy as np

# Assuming you have n vectors of dimension d in a numpy array
big_data = np.random.rand(800000, 1280).astype('float32')

index = faiss.index_factory(1280, "IVF2048,Flat")
index.train(big_data)

# Divide the dataset into smaller parts
batch_size = 1000000
start_idx = 0
batches = []
while start_idx < len(big_data):
    end_idx = min(start_idx + batch_size, len(big_data))
    batches.append(big_data[start_idx:end_idx])
    start_idx = end_idx

def search(query_vec, k):
    top_distance = []
    top_indexes = []

    for batch in batches:
        index.add(batch)
        distance, indexes = index.search(query_vec, k)
        index.reset()

        if top_indexes:
            merged_distance = np.hstack((top_distance, distance))
            merged_indexes = np.hstack((top_indexes, indexes))
            new_sort_indices = np.argsort(merged_distance)[:, :k]
            top_distance = np.array([merged_distance[i, new_sort_indices[i]] for i in range(merged_distance.shape[0])])
            top_indexes = np.array([merged_indexes[i, new_sort_indices[i]] for i in range(merged_indexes.shape[0])])
        else:
            top_distance = distance
            top_indexes = indexes

    return top_distance, top_indexes

query_vectors = np.random.rand(10, 1280).astype('float32')  # Generate 10 random query vectors
k = 10  # Number of nearest neighbors to find

distance,indexes = search(query_vectors, k)

print("Distances:\n", distance)
print("Indexes:\n", indexes)

INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.


KeyboardInterrupt: 

In [2]:
import faiss
import numpy as np

big_data = np.random.rand(80000, 1280).astype('float32')

coarse_quantizer = faiss.IndexFlatL2(1280)
index = faiss.IndexIVFFlat(coarse_quantizer, 1280, 1024)
index.train(big_data)

# Divide the dataset into smaller parts and save them on disk
batch_size = 10000
start_idx = 0
part = 0
while start_idx < len(big_data):
    end_idx = min(start_idx + batch_size, len(big_data))
    batch = big_data[start_idx:end_idx]
    
    file_name = f'batch_{part}.npy'
    np.save(file_name, batch)
    start_idx = end_idx
    part += 1

index.make_direct_map()
faiss.write_index(index, "batch.idx")


In [10]:
import sys
sys.getsizeof(index)


48

In [14]:

index = faiss.read_index("batch.idx")
part = 0
file_name = f'batch_{part}.npy'
batch = np.load(file_name)
index.add(batch)

query_vectors = np.random.rand(2, 1280).astype('float32')  # Generate 10 random query vectors
k = 5  # Number of nearest neighbors to find
distance, indexes = index.search(query_vectors, k)

In [16]:
print(distance)
print(indexes)

[[196.58798 198.8354  200.25943 200.42883 201.83769]
 [204.06442 207.50412 207.85512 208.13489 208.60117]]
[[6971 5974 1396 7973 1482]
 [2929 8690 9727 2723 1653]]


In [12]:
import os
import glob
import time

def search(index, query_vec, k, prefix):

    files = glob.glob(os.path.join("", prefix + "_*.npy"))
    
    top_distance = []
    top_indexes = []

    for file_name in files:
        batch = np.load(file_name)

        index.add(batch)
        distance, indexes = index.search(query_vec, k)
        index.reset()
        
        if len(top_distance) == 0:
            top_distance = distance
            top_indexes = indexes
        else:
            merged_distance = np.hstack((top_distance, distance))
            merged_indexes = np.hstack((top_indexes, indexes))
            new_sort_indices = np.argsort(merged_distance)[:, :k]
            top_distance = np.array([merged_distance[i, new_sort_indices[i]] for i in range(merged_distance.shape[0])])
            top_indexes = np.array([merged_indexes[i, new_sort_indices[i]] for i in range(merged_indexes.shape[0])])

    return top_distance, top_indexes

index = faiss.read_index("batch.idx")
#query_vectors = np.random.rand(2, 1280).astype('float32')  # Generate 10 random query vectors
query_vectors = big_data[0:2,:]
k = 5  # Number of nearest neighbors to find

st = time.time()
distance, indexes = search(index, query_vectors, k, "batch")
print(time.time()-st)

print("Distances:\n", distance)
print("Indexes:\n", indexes)

0.24608397483825684
Distances:
 [[  0.      190.92966 194.85794 195.5722  195.98907]
 [  0.      187.3742  188.67325 188.97668 189.07774]]
Indexes:
 [[   0 1943 6802  541 3230]
 [   1 9048 5361  243  326]]


In [11]:
big_data[0:2,:].shape

(2, 1280)

In [10]:
query_vectors.shape

(2, 1280)